<a href="https://colab.research.google.com/github/GandzyTM/tg_bot/blob/main/Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, BatchNormalization, AveragePooling2D
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
import tensorflow_datasets as tfds  # pip install tensorflow-datasets
import tensorflow as tf
import logging
import numpy as np
import time


In [2]:
model = Sequential()
model.add(Dense(4, input_dim=2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate=0.1))

In [3]:
X = np.array([[0,0], [0,1], [1,0], [1,1]])
y = np.array([[0], [1], [1], [0]])
model.fit(X, y, batch_size=1, epochs=1000, verbose=0)

In [4]:
def mnist_make_model(image_w: int, image_h: int):
   # Neural network model
   model = Sequential()
   model.add(Dense(784, activation='relu', input_shape=(image_w*image_h,)))
   model.add(Dense(10, activation='softmax'))
   model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])
   return model

def mnist_make_model2(image_w: int, image_h: int):
   # Neural network model
   model = Sequential()
   model.add(Dense(1024, activation='relu', input_shape=(image_w*image_h,)))
   model.add(Dropout(0.2))  # rate 0.2 - set 20% of inputs to zero
   model.add(Dense(1024, activation='relu'))
   model.add(Dropout(0.2))
   model.add(Dense(10, activation='softmax'))
   model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])
   return model

In [5]:
def mnist_mlp_train(model):
   (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
   # x_train: 60000x28x28 array, x_test: 10000x28x28 array
   image_size = x_train.shape[1]
   train_data = x_train.reshape(x_train.shape[0], image_size*image_size)
   test_data = x_test.reshape(x_test.shape[0], image_size*image_size)
   train_data = train_data.astype('float32')
   test_data = test_data.astype('float32')
   train_data /= 255.0
   test_data /= 255.0
   # encode the labels - we have 10 output classes
   # 3 -> [0 0 0 1 0 0 0 0 0 0], 5 -> [0 0 0 0 0 1 0 0 0 0]
   num_classes = 10
   train_labels_cat = keras.utils.to_categorical(y_train, num_classes)
   test_labels_cat = keras.utils.to_categorical(y_test, num_classes)
   print("Training the network...")
   t_start = time.time()
   # Start training the network
   model.fit(train_data, train_labels_cat, epochs=9, batch_size=64, verbose=1, validation_data=(test_data, test_labels_cat))

In [31]:
model = mnist_make_model(image_w=28, image_h=28)
mnist_mlp_train(model)
model.save('mlp_digits_28x28.h5')

Training the network...
Epoch 1/9
938/938 [==============================] - 11s 12ms/step - loss: 0.2061 - accuracy: 0.9382 - val_loss: 0.1119 - val_accuracy: 0.9641
Epoch 2/9
938/938 [==============================] - 10s 11ms/step - loss: 0.0846 - accuracy: 0.9749 - val_loss: 0.0759 - val_accuracy: 0.9768
Epoch 3/9
938/938 [==============================] - 10s 11ms/step - loss: 0.0585 - accuracy: 0.9826 - val_loss: 0.0795 - val_accuracy: 0.9755
Epoch 4/9
938/938 [==============================] - 10s 11ms/step - loss: 0.0429 - accuracy: 0.9876 - val_loss: 0.0754 - val_accuracy: 0.9785
Epoch 5/9
938/938 [==============================] - 10s 11ms/step - loss: 0.0320 - accuracy: 0.9903 - val_loss: 0.0667 - val_accuracy: 0.9831
Epoch 6/9
938/938 [==============================] - 10s 11ms/step - loss: 0.0252 - accuracy: 0.9927 - val_loss: 0.0728 - val_accuracy: 0.9810
Epoch 7/9
938/938 [==============================] - 10s 11ms/step - loss: 0.0196 - accuracy: 0.9943 - val_loss: 0.077

In [6]:
def mlp_digits_predict(model, image_file):
   image_size = 28
   img = keras.preprocessing.image.load_img(image_file, target_size=(image_size, image_size), color_mode='grayscale')
   img_arr = np.expand_dims(img, axis=0)
   img_arr = 1 - img_arr/255.0
   img_arr = img_arr.reshape((1, image_size*image_size))
   result = np.argmax(model.predict([img_arr]), axis=1)
   return result[0]

In [33]:
model = tf.keras.models.load_model('mlp_digits_28x28.h5')
print(mlp_digits_predict(model, 'digit_0.png'))
print(mlp_digits_predict(model, 'digit_1.png'))
print(mlp_digits_predict(model, 'digit_3.png'))
print(mlp_digits_predict(model, 'digit_8.png'))
print(mlp_digits_predict(model, 'digit_9.png'))

0
8
3
8
4


Сверточная сеть:

In [7]:
def mnist_cnn_model():
   image_size = 28
   num_channels = 1  # 1 for grayscale images
   num_classes = 10  # Number of outputs
   model = Sequential()
   model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', 
            padding='same', input_shape=(image_size, image_size, num_channels)))
   model.add(MaxPooling2D(pool_size=(2, 2)))
   model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu',
            padding='same'))
   model.add(MaxPooling2D(pool_size=(2, 2)))
   model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu',
            padding='same'))
   model.add(MaxPooling2D(pool_size=(2, 2)))
   model.add(Flatten())
   # Densely connected layers
   model.add(Dense(128, activation='relu'))
   # Output layer
   model.add(Dense(num_classes, activation='softmax'))
   model.compile(optimizer=Adam(), loss='categorical_crossentropy',
            metrics=['accuracy'])
   return model

In [8]:
def mnist_cnn_train(model):
   (train_digits, train_labels), (test_digits, test_labels) = keras.datasets.mnist.load_data()

   # Get image size
   image_size = 28
   num_channels = 1  # 1 for grayscale images

   # re-shape and re-scale the images data
   train_data = np.reshape(train_digits, (train_digits.shape[0], image_size, image_size, num_channels))
   train_data = train_data.astype('float32') / 255.0
   # encode the labels - we have 10 output classes
   # 3 -> [0 0 0 1 0 0 0 0 0 0], 5 -> [0 0 0 0 0 1 0 0 0 0]
   num_classes = 10
   train_labels_cat = keras.utils.to_categorical(train_labels, num_classes)

   # re-shape and re-scale the images validation data
   val_data = np.reshape(test_digits, (test_digits.shape[0], image_size, image_size, num_channels))
   val_data = val_data.astype('float32') / 255.0
   # encode the labels - we have 10 output classes
   val_labels_cat = keras.utils.to_categorical(test_labels, num_classes)

   print("Training the network...")
   t_start = time.time()

   # Start training the network
   model.fit(train_data, train_labels_cat, epochs=8, batch_size=64,
        validation_data=(val_data, val_labels_cat))

   print("Done, dT:", time.time() - t_start)

   return model

In [9]:
model = mnist_cnn_model()
mnist_cnn_train(model)
model.save('cnn_digits_28x28.h5')

11493376/11490434 [==============================] - 0s 0us/step
Training the network...
Epoch 1/8
938/938 [==============================] - 87s 92ms/step - loss: 0.1625 - accuracy: 0.9492 - val_loss: 0.0418 - val_accuracy: 0.9861
Epoch 2/8
938/938 [==============================] - 86s 92ms/step - loss: 0.0452 - accuracy: 0.9865 - val_loss: 0.0427 - val_accuracy: 0.9873
Epoch 3/8
938/938 [==============================] - 86s 91ms/step - loss: 0.0315 - accuracy: 0.9901 - val_loss: 0.0256 - val_accuracy: 0.9918
Epoch 4/8
938/938 [==============================] - 92s 98ms/step - loss: 0.0263 - accuracy: 0.9919 - val_loss: 0.0322 - val_accuracy: 0.9907
Epoch 5/8
938/938 [==============================] - 92s 98ms/step - loss: 0.0200 - accuracy: 0.9938 - val_loss: 0.0286 - val_accuracy: 0.9910
Epoch 6/8
938/938 [==============================] - 92s 98ms/step - loss: 0.0166 - accuracy: 0.9948 - val_loss: 0.0312 - val_accuracy: 0.9914
Epoch 7/8
938/938 [==============================] - 

In [11]:
def cnn_digits_predict(model, image_file):
   image_size = 28
   img = keras.preprocessing.image.load_img(image_file, 
target_size=(image_size, image_size), color_mode='grayscale')
   img_arr = np.expand_dims(img, axis=0)
   img_arr = 1 - img_arr/255.0
   img_arr = img_arr.reshape((1, 28, 28, 1))

   result = np.argmax(model.predict([img_arr]), axis=1)
   return result[0]

In [13]:
model = tf.keras.models.load_model('cnn_digits_28x28.h5')
print(cnn_digits_predict(model, 'digit_0.png'))
print(cnn_digits_predict(model, 'digit_1.png'))
print(cnn_digits_predict(model, 'digit_3.png'))
print(cnn_digits_predict(model, 'digit_8.png'))
print(cnn_digits_predict(model, 'digit_9.png'))
print(cnn_digits_predict(model, 'digit_5.png'))
print(cnn_digits_predict(model, 'digit_2.png'))
print(cnn_digits_predict(model, 'digit_34.png'))

0
1
3
8
9
5
2
4
